اول حاجه هنحط الداتا في مونجو دي بي 

In [ ]:
#this is new DAG
import json
from pymongo import MongoClient

#  الاتصال بـ MongoDB
client = MongoClient("mongodb://localhost:27017/")

#  اختيار Database و Collection
db = client["nyc_data"]
collection = db["nyc_traffic"]

# (اختياري) تفريغ الكوليكشن لو بتجرب
# collection.delete_many({})

#  قراءة ملف الـ JSON
with open("nyc_traffic.json", "r") as f:
    data = json.load(f)

# تأكيد إن الداتا Array
assert isinstance(data, list), "JSON file must contain a list of documents"

#  إدخال الدhتا
collection.insert_many(data)

print(f" Inserted {len(data)} documents into MongoDB successfully")


✅ Inserted 41362 documents into MongoDB successfully


حولنا الداتا بتتاعت الشوارع من ملف سي اي في اي  ملف  parquet
ودي عشان ننوع من مصارد الداتا الي جايلنا 

In [1]:

import pandas as pd 
data = pd.read_csv("Centerline.csv")
data.to_parquet("streets_data.parquet")

انشاء اول تاسك في الداج وده هنعمل كونكت مع الاس 3 و هنكريت باكت 
ونعمل فيها 2 فولدر وبعدين نرفع الرو داتا فيها 
1- هنجيب الداتا  من مونجو دي بي ونرفعها علي هئيه جيسون فايل 
2- هنجيب الفايل البركيه ونرفعه 
3- هنجيب الفايل الثالث والاخير ونرفعه الي هو csv 

In [12]:
from pymongo import MongoClient
import json
import boto3

# اتصال Mongo
client = MongoClient("mongodb://localhost:27017")
db = client["nyc_data"]
collection = db["nyc_traffic"]

#-----------------------------------------------
# connect with s3 
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9010',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123'
)
Bucket="datalake"

#------------------------------------------------
# create bucket لو مش موجود 
try:
     s3.create_bucket(Bucket="datalake")
except:
     pass

#------------------------------------------------
# create 2 folder (raw , processed)
try:
     s3.put_object(Bucket="datalake", Key="raw/")
     s3.put_object(Bucket="datalake", Key="processed/")
except:
     pass

#----------------------------------------------------------
# upload first file to s3 (json file)
cursor = collection.find({}, {"_id": 0})

data = list(cursor)          # in-memory
json_bytes = json.dumps(
    data,
    ensure_ascii=False
).encode("utf-8")

#load to s3
s3.Bucket("datalake").put_object(
    Key="raw/mongo_traffic.json",
    Body=json_bytes,
    ContentType="application/json"
)


#---------------------------------------------------==
# upload second file csv to s3
s3_client = boto3.client(
    's3',
    endpoint_url='http://localhost:9010',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123'
)


s3_client.upload_file(
    Filename="Motor_Vehicle_Collisions_Crashes.csv",  # المسار المحلي للفايل
    Bucket="datalake",                    # اسم الباكيت في S3/MinIO
    Key="raw/Motor_Vehicle_Collisions_Crashes.csv",    # المسار داخل الباكيت
    ExtraArgs={"ContentType": "text/csv"}   # دي زي ميتا داتا بعرفه ايه الكونتنت الي جوا الفايل ده 

)

#--------------------------------------
# upload the last file (parquet file )

s3_client.upload_file(
    Filename="streets_data.parquet",  # المسار المحلي للفايل
    Bucket="datalake",                    # اسم الباكيت في S3/MinIO
    Key="raw/streets_data.parquet" ,   # المسار داخل الباكيت
    ExtraArgs={"ContentType": "application/octet-stream"}  # دي زي ميتا داتا بعرفه ايه الكونتنت الي جوا الفايل ده 

)




التاسك التاني هنجيب الداتا بتاعت الجيسون فايل ونحولها لداتا فريم 
دا بستخدام باي سبارك و نعمل عليها بروسيسنج وبعدين نرفعها علي س3 تاني بس المره ده في فولدر اسمه بروسيس داتا 


In [ ]:
# task 2
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("transform1")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)

#spark.sparkContext.setLogLevel("WARN")
# هنجيب الداتا بقا ونحولها لداتا فريم

df = spark.read.json(
    "s3a://datalake/raw/mongo_traffic.json"
)

#df.show(20)
#----------------------------------------------------------
# هنرجع الداتا تاني في الفولدر الي هو processed
#  وهنخلي الفايل يبقا parquet  عشان يحتفظ ب الاسيكما احسن من csv 

df.write \
    .mode("overwrite") \
    .parquet("s3a://datalake/processed/mongo_traffic")




التاسك التاني 
ودي عباره عن اننا هنجيب الفايل الي هو csv 
وهنبدء نتعامل معاه علي pyspark 
 بحيث اننا نعمل بروسيسنج لداتا الي فيه بما يتناسب مع احتياجتنا 
 وفي الاخر نرفع الفايل علي هئيه بركيه فايل علي s3
جوا الفولدر الي اسمه processed

In [1]:
# task 3
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("transform2")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)

# هنجيب الداتا  ونحولها لداتا فريم

df = spark.read.csv(
    "s3a://datalake/raw/Motor_Vehicle_Collisions_Crashes.csv"
)

df.show(20)
#----------------------------------------------------------
# هنرجع الداتا تاني في الفولدر الي هو processed
#  وهنخلي الفايل يبقا parquet  عشان يحتفظ ب الاسيكما احسن من csv 

# df.write \
#     .mode("overwrite") \
#     .parquet("s3a://datalake/processed/mongo_traffic")




Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/02 02:49:25 WARN Utils: Your hostname, refat, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
26/01/02 02:49:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ahmed-refat/miniconda3/envs/spark/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ahmed-refat/.ivy2.5.2/cache
The jars for the packages stored in: /home/ahmed-refat/.ivy2.5.2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0ae4c4b4-9da5-4d19-87aa-5f152311aa37;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.4.0 in central
	found software.amazon.awssdk#bundle;2.23.19 in central
	found org.wildfly.openssl#wildfly-open

+----------+----------+---------+--------+---------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|       _c0|       _c1|      _c2|     _c3|      _c4|       _c5|                 _c6|                 _c7|                 _c8|                 _c9|                _c10|                _c11|                _c12|                _c13|                _c14|                _c15|                _c16|                _c17|                _c18|                _c19|                _c20|                _c21|                _c22|        _c23|                _c24|          